In [1]:
import pandas as pd
import requests
import json
import warnings
from time import sleep

warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.mode.chained_assignment = None

In [2]:
# df = pd.read_excel('Share of Voice - Grupos_TRATADO.xlsx', sheet_name='Planilha1', engine='openpyxl')
# df['status_proc'] = ''
# df['id_api'] = ''

In [41]:
df = pd.read_csv('Share of Voice - Grupos_TRATADO_ERRORS_v3.csv')
df_ok1 = pd.read_csv('Share of Voice - Grupos_PROCESSED_v1.csv')
df_ok2 = pd.read_csv('Share of Voice - Grupos_PROCESSED_v2.csv')
df_ok3 = pd.read_csv('Share of Voice - Grupos_PROCESSED_v3.csv')
df_ok4 = pd.read_csv('Share of Voice - Grupos_PROCESSED_v4.csv')
df_ok5 = pd.read_csv('Share of Voice - Grupos_PROCESSED_v5.csv')

In [42]:
ids_excluir = pd.concat([df_ok1['cust_id'], df_ok2['cust_id'], df_ok3['cust_id'], df_ok4['cust_id'], df_ok5['cust_id']]).unique()

In [43]:
df = df[~df['cust_id'].isin(ids_excluir)]
df.reset_index(drop=True, inplace=True)

In [44]:
df

,cust_id,status_cust_id,class_atual,deals_group_on_wpp,link,tipo_link,status_proc,id_api
0,1299080285,Confirmado,KAM - top 1000,Sim,https://achadinhosimperdiveis.com.br/achadinho...,Grupo,ERROR,NaN
1,1374758175,Confirmado,KAM - top 1000,Sim,https://sndflw.com/i/grupospromotop,Grupo,ERROR,NaN
2,2313173804,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/EexXms38M3O3yhSsIcmc6r,Grupo,ERROR,NaN
3,744048622,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/BZpAuh2smnSBgAMWUIr8it,Grupo,ERROR,NaN
4,2221972417,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/ESRK7K8eTez9n4tsWi8C...,Grupo,ERROR,NaN
5,147294155,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/GlZbsfe4H0q7GdhETjruhL,Grupo,ERROR,NaN
6,2018286552,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/BLEQ9LQyRqnHpZ2bVWmBeT,Grupo,ERROR,NaN
7,431811888,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/Lf0hxVaeCxNDBvw2ydy6PV,Grupo,ERROR,NaN
8,470806864,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/ED3SWc4c1ZT2oeACRALS...,Grupo,ERROR,NaN
9,66404218,Confirmado,KAM - top 1000,Sim,https://chat.whatsapp.com/CLsmnQK13dGCCWOlIJ3k...,Grupo,ERROR,NaN


In [45]:
url = "http://localhost:3000"
endpoint_listar_grupos = "/api/default/groups/join"

In [46]:
for x in range(len(df)):
    if df.tipo_link[x] == 'Grupo':
        print(f"Processing row {x} with link: {df.link[x]}")
        try:
            payload = json.dumps({
                "code": str(str(df.link[x]).split('?')[0])
            })
            headers = {
                'Content-Type': 'application/json'
            }
            response = requests.request("POST", url + endpoint_listar_grupos, headers=headers, data=payload)
            resp_json = response.json()
            df.id_api[x] = resp_json['id']
            df.status_proc[x] = 'processed'
        except Exception as e:
            print(f"Error processing row {x}: {e}")
            print(f"Response JSON: {resp_json}")
            df.status_proc[x] = 'ERROR'
    elif df.tipo_link[x] == 'Conversa' or df.tipo_link[x] == 'Telegram' or df.tipo_link[x] == 'Link Quebrado' or df.tipo_link[x] == 'Link Invalido' or df.tipo_link[x] == 'Sem Link':
        df.status_proc[x] = 'ERROR'
    elif df.tipo_link[x] == 'Canal':
        df.status_proc[x] = 'to_process'
    sleep(2)

Processing row 0 with link: https://achadinhosimperdiveis.com.br/achadinho-grupo
Error processing row 0: 'id'
Response JSON: {'statusCode': 500, 'timestamp': '2025-11-06T13:04:13.284Z', 'exception': {'message': '2 UNKNOWN: info query returned status 400: bad-request', 'code': 2, 'details': 'info query returned status 400: bad-request', 'metadata': {}, 'name': 'Error', 'stack': 'Error: 2 UNKNOWN: info query returned status 400: bad-request\n    at callErrorFromStatus (/app/node_modules/@grpc/grpc-js/build/src/call.js:32:19)\n    at Object.onReceiveStatus (/app/node_modules/@grpc/grpc-js/build/src/client.js:193:76)\n    at Object.onReceiveStatus (/app/node_modules/@grpc/grpc-js/build/src/client-interceptors.js:361:141)\n    at Object.onReceiveStatus (/app/node_modules/@grpc/grpc-js/build/src/client-interceptors.js:324:181)\n    at /app/node_modules/@grpc/grpc-js/build/src/resolving-call.js:135:78\n    at process.processTicksAndRejections (node:internal/process/task_queues:85:11)\nfor cal

In [47]:
df.status_proc.unique()

array(['ERROR'], dtype=object)

In [40]:
df.to_csv('Share of Voice - Grupos_TRATADO_ERRORS_v3.csv', index=False)

In [33]:
df.query("status_proc == 'processed'").to_csv('Share of Voice - Grupos_PROCESSED_v5.csv', index=False)